Batch Normalization
- Internal Covariate Shift 방지
    - Internal Covariate Shift: layer들을 거치며 input으로 들어온 distribution이 변형되는 현상
- 각 layer의 input으로 들어오는 distribution을 계속 normalize하여 distribution을 항상 일정하게 유지

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

tf.enable_eager_execution()

In [2]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [3]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [4]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

layer -> normalization -> activation 순서대로 주로 사용

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [7]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [9]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.9669, train_loss: 0.34448981, train_accuracy: 0.3359, test_Accuracy: 0.2034
Epoch: [ 0] [    1/  468] time: 2.6737, train_loss: 0.28602469, train_accuracy: 0.4375, test_Accuracy: 0.4012
Epoch: [ 0] [    2/  468] time: 3.3121, train_loss: 0.30957770, train_accuracy: 0.4766, test_Accuracy: 0.5028
Epoch: [ 0] [    3/  468] time: 3.9484, train_loss: 0.40494463, train_accuracy: 0.5625, test_Accuracy: 0.5407
Epoch: [ 0] [    4/  468] time: 4.4919, train_loss: 0.31158420, train_accuracy: 0.6484, test_Accuracy: 0.5540
Epoch: [ 0] [    5/  468] time: 5.1453, train_loss: 0.28768265, train_accuracy: 0.5312, test_Accuracy: 0.5658
Epoch: [ 0] [    6/  46

Epoch: [ 0] [   71/  468] time: 53.1150, train_loss: 0.15094507, train_accuracy: 0.9609, test_Accuracy: 0.9188
Epoch: [ 0] [   72/  468] time: 53.6980, train_loss: 0.13808447, train_accuracy: 0.9453, test_Accuracy: 0.9192
Epoch: [ 0] [   73/  468] time: 54.3036, train_loss: 0.11257483, train_accuracy: 0.9453, test_Accuracy: 0.9217
Epoch: [ 0] [   74/  468] time: 55.0030, train_loss: 0.11501026, train_accuracy: 0.9453, test_Accuracy: 0.9230
Epoch: [ 0] [   75/  468] time: 55.7381, train_loss: 0.09909129, train_accuracy: 0.9297, test_Accuracy: 0.9229
Epoch: [ 0] [   76/  468] time: 56.4333, train_loss: 0.12260897, train_accuracy: 0.9062, test_Accuracy: 0.9219
Epoch: [ 0] [   77/  468] time: 57.1642, train_loss: 0.22853050, train_accuracy: 0.9062, test_Accuracy: 0.9235
Epoch: [ 0] [   78/  468] time: 57.8860, train_loss: 0.20877741, train_accuracy: 0.9375, test_Accuracy: 0.9259
Epoch: [ 0] [   79/  468] time: 58.6077, train_loss: 0.09450158, train_accuracy: 0.9297, test_Accuracy: 0.9290
E

Epoch: [ 0] [  145/  468] time: 105.8710, train_loss: 0.11970945, train_accuracy: 0.9609, test_Accuracy: 0.9574
Epoch: [ 0] [  146/  468] time: 106.7129, train_loss: 0.17301884, train_accuracy: 0.9375, test_Accuracy: 0.9577
Epoch: [ 0] [  147/  468] time: 107.4393, train_loss: 0.11368824, train_accuracy: 0.9688, test_Accuracy: 0.9572
Epoch: [ 0] [  148/  468] time: 108.0905, train_loss: 0.11048062, train_accuracy: 0.9609, test_Accuracy: 0.9567
Epoch: [ 0] [  149/  468] time: 108.7694, train_loss: 0.05946391, train_accuracy: 0.9766, test_Accuracy: 0.9571
Epoch: [ 0] [  150/  468] time: 109.4595, train_loss: 0.06160142, train_accuracy: 0.9609, test_Accuracy: 0.9569
Epoch: [ 0] [  151/  468] time: 110.0633, train_loss: 0.08670944, train_accuracy: 0.9609, test_Accuracy: 0.9566
Epoch: [ 0] [  152/  468] time: 110.6643, train_loss: 0.06905816, train_accuracy: 0.9766, test_Accuracy: 0.9566
Epoch: [ 0] [  153/  468] time: 111.2336, train_loss: 0.11981064, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  219/  468] time: 157.9697, train_loss: 0.09553187, train_accuracy: 0.9453, test_Accuracy: 0.9555
Epoch: [ 0] [  220/  468] time: 158.5039, train_loss: 0.06545785, train_accuracy: 0.9766, test_Accuracy: 0.9567
Epoch: [ 0] [  221/  468] time: 159.0682, train_loss: 0.13392743, train_accuracy: 0.9688, test_Accuracy: 0.9575
Epoch: [ 0] [  222/  468] time: 159.6444, train_loss: 0.24497330, train_accuracy: 0.9141, test_Accuracy: 0.9572
Epoch: [ 0] [  223/  468] time: 160.2307, train_loss: 0.11474501, train_accuracy: 0.9453, test_Accuracy: 0.9579
Epoch: [ 0] [  224/  468] time: 160.9047, train_loss: 0.07828446, train_accuracy: 0.9609, test_Accuracy: 0.9584
Epoch: [ 0] [  225/  468] time: 161.6048, train_loss: 0.04899161, train_accuracy: 0.9844, test_Accuracy: 0.9582
Epoch: [ 0] [  226/  468] time: 162.2738, train_loss: 0.09201889, train_accuracy: 0.9531, test_Accuracy: 0.9574
Epoch: [ 0] [  227/  468] time: 162.9293, train_loss: 0.07065877, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  293/  468] time: 204.2789, train_loss: 0.11932384, train_accuracy: 0.9688, test_Accuracy: 0.9656
Epoch: [ 0] [  294/  468] time: 204.9139, train_loss: 0.07273561, train_accuracy: 0.9766, test_Accuracy: 0.9642
Epoch: [ 0] [  295/  468] time: 205.5429, train_loss: 0.14272667, train_accuracy: 0.9453, test_Accuracy: 0.9628
Epoch: [ 0] [  296/  468] time: 206.1932, train_loss: 0.12629469, train_accuracy: 0.9531, test_Accuracy: 0.9619
Epoch: [ 0] [  297/  468] time: 206.8133, train_loss: 0.11692776, train_accuracy: 0.9609, test_Accuracy: 0.9608
Epoch: [ 0] [  298/  468] time: 207.4299, train_loss: 0.06611878, train_accuracy: 0.9609, test_Accuracy: 0.9589
Epoch: [ 0] [  299/  468] time: 208.0297, train_loss: 0.01869546, train_accuracy: 0.9922, test_Accuracy: 0.9574
Epoch: [ 0] [  300/  468] time: 208.6235, train_loss: 0.08605145, train_accuracy: 0.9766, test_Accuracy: 0.9563
Epoch: [ 0] [  301/  468] time: 209.2471, train_loss: 0.10139302, train_accuracy: 0.9453, test_Accuracy:

Epoch: [ 0] [  367/  468] time: 253.9539, train_loss: 0.04698670, train_accuracy: 0.9922, test_Accuracy: 0.9678
Epoch: [ 0] [  368/  468] time: 254.6435, train_loss: 0.05357844, train_accuracy: 0.9844, test_Accuracy: 0.9680
Epoch: [ 0] [  369/  468] time: 255.3117, train_loss: 0.06656422, train_accuracy: 0.9844, test_Accuracy: 0.9682
Epoch: [ 0] [  370/  468] time: 256.0208, train_loss: 0.09707104, train_accuracy: 0.9844, test_Accuracy: 0.9689
Epoch: [ 0] [  371/  468] time: 256.7112, train_loss: 0.06461936, train_accuracy: 0.9766, test_Accuracy: 0.9697
Epoch: [ 0] [  372/  468] time: 257.4126, train_loss: 0.05323647, train_accuracy: 1.0000, test_Accuracy: 0.9697
Epoch: [ 0] [  373/  468] time: 258.1379, train_loss: 0.13323966, train_accuracy: 0.9688, test_Accuracy: 0.9685
Epoch: [ 0] [  374/  468] time: 258.8456, train_loss: 0.09162847, train_accuracy: 0.9688, test_Accuracy: 0.9687
Epoch: [ 0] [  375/  468] time: 259.5031, train_loss: 0.09422316, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  441/  468] time: 299.4215, train_loss: 0.07208781, train_accuracy: 0.9844, test_Accuracy: 0.9679
Epoch: [ 0] [  442/  468] time: 300.1540, train_loss: 0.09336331, train_accuracy: 0.9766, test_Accuracy: 0.9676
Epoch: [ 0] [  443/  468] time: 300.8329, train_loss: 0.03429845, train_accuracy: 0.9844, test_Accuracy: 0.9679
Epoch: [ 0] [  444/  468] time: 301.5039, train_loss: 0.05247834, train_accuracy: 0.9844, test_Accuracy: 0.9681
Epoch: [ 0] [  445/  468] time: 302.0846, train_loss: 0.05392680, train_accuracy: 0.9766, test_Accuracy: 0.9684
Epoch: [ 0] [  446/  468] time: 302.6448, train_loss: 0.05466738, train_accuracy: 0.9688, test_Accuracy: 0.9682
Epoch: [ 0] [  447/  468] time: 303.2258, train_loss: 0.03669006, train_accuracy: 0.9688, test_Accuracy: 0.9678
Epoch: [ 0] [  448/  468] time: 303.9126, train_loss: 0.04871600, train_accuracy: 0.9844, test_Accuracy: 0.9671
Epoch: [ 0] [  449/  468] time: 304.5375, train_loss: 0.03917769, train_accuracy: 0.9844, test_Accuracy: